# Langchain crash course

In [79]:
#from secret_key import openapi_key
from secret_key import hugging_token
import os
#os.environ['OPENAI_API_KEY'] = openapi_key
os.environ['HUGGINGFACEHUB_API_TOKEN'] = hugging_token

#import os
#os.environ['OPENAI_API_KEY'] = "key"

## LLMs

In [80]:
#!pip install langchain

In [81]:
#!pip install huggingface
#!pip install huggingface_hub

In [83]:
# for the Hugginface Hub API -> no local model
# for the localmodel the HuggingFacePipeline should be used:
# reffer to https://python.langchain.com/docs/integrations/providers/huggingface
# another good block: https://blog.futuresmart.ai/integrating-llama-2-with-hugging-face-and-langchain
from langchain import HuggingFaceHub, LLMChain

#hub_llm = HuggingFaceHub(repo_id="meta-llama/Llama-2-13b-chat-hf") # repo gated and access awayted
hub_llm_zephyr = HuggingFaceHub(repo_id="HuggingFaceH4/zephyr-7b-alpha")
hub_llm_mistral = HuggingFaceHub(repo_id="mistralai/Mistral-7B-v0.1")
name1 = hub_llm_zephyr.predict("I want to open a restaurant for Indian food. Suggest a fancy name for this.")
print(name1)

name2 = hub_llm_mistral.predict("I want to open a restaurant for Indian food. Suggest a fancy name for this.")
print(name2)

/Users/philipbuchbender/opt/anaconda3/envs/ml_pytho/lib/python3.9/site-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '0.19.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)
/Users/philipbuchbender/opt/anaconda3/envs/ml_pytho/lib/python3.9/site-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '0.19.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/


The name should be unique and catchy.

1. Spice Odyssey



I want to open a restaurant for Indian food. Suggest a fancy name for this.


In [ ]:
#!pip install openai

In [ ]:
# for the OpenAI api
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)
name = llm.predict("I want to open a restaurant for Indian food. Suggest a fancy name for this.")
print(name)

In [ ]:
llm("I want to open a restaurant for Indian food. Suggest a fancy name for this.")

'\n\nSaffron & Spice.'

## Prompt Templates

In [ ]:
# with huggingfacehub models
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Could you suggest a fancy name for this."
)
p = prompt_template_name.format(cuisine="Italian")
print(p)

#name = hub_llm.predict(p)


I want to open a restaurant for Italian food. Could you suggest a fancy name for this.


In [ ]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fancy name for this."
)
p = prompt_template_name.format(cuisine="Italian")
print(p)

# print(llm.predict(p))

I want to open a restaurant for Italian food. Suggest a fency name for this.


## Chains

In [ ]:
# for hugginfacehub model
from langchain.chains import LLMChain

chain = LLMChain(llm=hub_llm_zephyr, prompt=prompt_template_name, verbose=True)
print(chain.run("Mexican"))

chain = LLMChain(llm=hub_llm_mistral, prompt=prompt_template_name, verbose=True)
print(chain.run("Mexican"))



> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for Mexican food. Suggest a fency name for this.

> Finished chain.

The name should be catchy and easy to remember.
It should also be unique and not


> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for Mexican food. Suggest a fency name for this.

> Finished chain.


I want to open a restaurant for Mexican food. Suggest a fency name for this


this seems false but in reallity this is the literal output from the hub model!!

In [ ]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
chain.run("Mexican")

'\n\n"La Casa de Sabor Mexicano"'

In [ ]:
chain = LLMChain(llm=llm, prompt=prompt_template_name, verbose=True)
chain.run("Mexican")



> Entering new  chain...
Prompt after formatting:
I want to open a restaurant for Mexican food. Suggest a fency name for this.

> Finished chain.


' \n\nTaco Tequila Palace'

In [ ]:
prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain_1 =LLMChain(llm=hub_llm_zephyr, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="""Suggest some menu items for {restaurant_name}"""
)

food_items_chain_1 = LLMChain(llm=hub_llm_zephyr, prompt=prompt_template_items, verbose=True)

In [ ]:
prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain_2 =LLMChain(llm=hub_llm_mistral, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="""Suggest some menu items for {restaurant_name}"""
)

food_items_chain_2 = LLMChain(llm=hub_llm_mistral, prompt=prompt_template_items, verbose=True)

In [ ]:
from langchain.chains import SimpleSequentialChain
chain1 = SimpleSequentialChain(chains = [name_chain_1, food_items_chain_1])

from langchain.chains import SimpleSequentialChain
chain2 = SimpleSequentialChain(chains = [name_chain_2, food_items_chain_2])

content = chain1.run("Indian")
print(content)

content = chain2.run("Indian")
print(content)



> Entering new LLMChain chain...
Prompt after formatting:
Suggest some menu items for 
The name should be catchy and easy to remember.

1. Spice Route


> Finished chain.
- Butter Chicken Tacos
- Chana Masala Quesadillas
- T


> Entering new LLMChain chain...
Prompt after formatting:
Suggest some menu items for 

I want to open a restaurant for Indian food. Suggest a fency name for this

> Finished chain.
 restaurant.

I want to open a restaurant for Indian food. Suggest a fency name


like we see here the model works fine but somehow does not work with the previous promt for some reason.

In [ ]:
llm = OpenAI(temperature=0.6)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="""Suggest some menu items for {restaurant_name}"""
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

#### Simple Sequential Chain

In [ ]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])

content = chain.run("Indian")
print(content)



- Chicken Tikka Masala
- Aloo Gobi
- Biryani
- Palak Paneer
- Chana Masala
- Samosas
- Naan
- Raita
- Korma
- Gulab Jamun
- Paneer Tikka
- Tandoori Chicken
- Vegetable Pakoras
- Mango Lassi
- Papadums


#### Sequential Chain

In [ ]:
# huggingface llms
hub_llm_zephyr = HuggingFaceHub(repo_id="HuggingFaceH4/zephyr-7b-alpha")
hub_llm_mistral = HuggingFaceHub(repo_id="mistralai/Mistral-7B-v0.1")

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain_zephyr =LLMChain(llm=hub_llm_zephyr, prompt=prompt_template_name, output_key="restaurant_name")
name_chain_mistral =LLMChain(llm=hub_llm_mistral, prompt=prompt_template_name, output_key="restaurant_name")


/Users/philipbuchbender/opt/anaconda3/envs/ml_pytho/lib/python3.9/site-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '0.19.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)
/Users/philipbuchbender/opt/anaconda3/envs/ml_pytho/lib/python3.9/site-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '0.19.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/

In [ ]:
hub_llm_zephyr = HuggingFaceHub(repo_id="HuggingFaceH4/zephyr-7b-alpha")
hub_llm_mistral = HuggingFaceHub(repo_id="mistralai/Mistral-7B-v0.1")

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)

food_items_chain_zephyr =LLMChain(llm=hub_llm_zephyr, prompt=prompt_template_items, output_key="menu_items")
food_items_chain_mistral =LLMChain(llm=hub_llm_mistral, prompt=prompt_template_items, output_key="menu_items")

/Users/philipbuchbender/opt/anaconda3/envs/ml_pytho/lib/python3.9/site-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '0.19.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)
/Users/philipbuchbender/opt/anaconda3/envs/ml_pytho/lib/python3.9/site-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '0.19.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/

In [ ]:
from langchain.chains import SequentialChain

chain_1 = SequentialChain(
    chains = [name_chain_zephyr, food_items_chain_zephyr],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', "menu_items"]
)

chain_2 = SequentialChain(
    chains = [name_chain_mistral, food_items_chain_mistral],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', "menu_items"]
)

In [ ]:
chain_1({"cuisine": "Indian"})

{'cuisine': 'Indian',
 'restaurant_name': '\nThe name should be catchy and easy to remember.\n\n1. Spice Route\n',
 'menu_items': ' Chicken Tikka Masala\n. Lamb Rogan Josh\n. Vegetable Biry'}

In [ ]:
chain_2({"cuisine": "Indian"})

{'cuisine': 'Indian',
 'restaurant_name': '\n\nI want to open a restaurant for Indian food. Suggest a fency name for this',
 'menu_items': '\n\nI want to open a restaurant for Indian food. Suggest a fency name for this'}

In [ ]:
llm = OpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

In [ ]:
llm = OpenAI(temperature=0.7)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)

food_items_chain =LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

In [ ]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', "menu_items"]
)

In [ ]:
chain({"cuisine": "Indian"})

{'cuisine': 'Indian',
 'restaurant_name': "\n\nMaharaja's Spice Palace",
 'menu_items': '\n\n- Chicken Tikka Masala\n- Vegetable Biryani\n- Lamb Rogan Josh\n- Paneer Makhani\n- Aloo Gobi\n- Palak Paneer\n- Chana Masala\n- Vegetable Korma\n- Samosas\n- Naan Bread\n- Papadums\n- Raita\n- Mango Lassi'}

## Agents

## those do not work with the hugginface api

In [ ]:
#!pip install google-search-results

In [ ]:
#make sure yoy have installed this package: pip install google-search-results
from secret_key import serpapi_key
os.environ['SERPAPI_API_KEY'] = "b0bf8059855d6a8c47f3ca881602a7297bed661fc2c99d411c127e56cf6b41db"#serpapi_key

#os.environ['SERPAPI_API_KEY'] = "add your serpapi key here"

#### serpapi and llm-math tool

In [ ]:
#!pip install --upgrade transformers huggingface_hub

In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import HuggingFaceHub

hub_llm_zephyr = HuggingFaceHub(repo_id="HuggingFaceH4/zephyr-7b-alpha")
hub_llm_mistral = HuggingFaceHub(repo_id="mistralai/Mistral-7B-v0.1")
hub_llm_prometheus = HuggingFaceHub(repo_id="kaist-ai/prometheus-13b-v1.0")
hub_llm_gpt = HuggingFaceHub(
    repo_id="gpt2",
    model_kwargs={'temperature': 0.1, 'max_length': 200}
)

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=hub_llm_gpt)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, hub_llm_gpt, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, handle_parsing_errors=True)

# Let's test it out!
agent.run("What was the GDP of US in 2022 plus 5?")

In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Let's test it out!
agent.run("What was the GDP of US in 2022 plus 5?")

#### Wikipedia and llm-math tool

In [ ]:
#!pip install wikipedia

In [ ]:
# install this package: pip install wikipedia
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import HuggingFaceHub

hub_llm_zephyr = HuggingFaceHub(repo_id="HuggingFaceH4/zephyr-7b-alpha")
hub_llm_mistral = HuggingFaceHub(repo_id="mistralai/Mistral-7B-v0.1")
hub_llm_prometheus = HuggingFaceHub(repo_id="kaist-ai/prometheus-13b-v1.0")

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["wikipedia", "llm-math"], llm=hub_llm_mistral, handle_parsing_errors=True)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(
    tools, 
    hub_llm_mistral, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True,
    handle_parsing_errors=True
)

# Let's test it out!
agent.run("When was Elon musk born? What is his age right now in 2023?")

In [ ]:
# install this package: pip install wikipedia

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True
)

# Let's test it out!
agent.run("When was Elon musk born? What is his age right now in 2023?")



> Entering new  chain...
 I need to find out when Elon Musk was born and then calculate his age.
Action: Wikipedia
Action Input: Elon Musk
Observation: Page: Elon Musk
Summary: Elon Reeve Musk ( EE-lon; born June 28, 1971) is a business magnate and investor. He is the founder, CEO, and chief engineer of SpaceX; angel investor, CEO, and product architect of Tesla, Inc.; owner, CTO, and chairman of Twitter; founder of the Boring Company and X Corp.; co-founder of Neuralink and OpenAI; and president of the philanthropic Musk Foundation. Musk is the wealthiest person in the world according to the Bloomberg Billionaires Index and Forbes's Real Time Billionaires list as of June 2023, primarily from his ownership stakes in Tesla and SpaceX, with an estimated net worth of around $225 billion according to Bloomberg and $230.2 billion according to Forbes.Musk was born in Pretoria, South Africa, and briefly attended the University of Pretoria before moving to Canada at age 18, acquiring citizen

'Elon Musk was born on June 28, 1971 and is currently 52 years old in 2023.'

## Memory

In [ ]:
# hugginface api
#hub_llm_zephyr = HuggingFaceHub(repo_id="HuggingFaceH4/zephyr-7b-alpha")
#hub_llm_mistral = HuggingFaceHub(repo_id="mistralai/Mistral-7B-v0.1")
hub_llm_prometheus = HuggingFaceHub(repo_id="kaist-ai/prometheus-13b-v1.0")
#hub_llm_gpt = HuggingFaceHub(
#    repo_id="gpt2",
#    model_kwargs={'temperature': 0.1, 'max_length': 200}
#)

chain = LLMChain(llm=hub_llm_prometheus,prompt=prompt_template_name)
name = chain.run("Mexican")
print(name)

In [ ]:
chain = LLMChain(llm=llm,prompt=prompt_template_name)
name = chain.run("Mexican")
print(name)



Taco Fiesta


In [ ]:
name = chain.run("Indian")
print(name)



Maharaja's Palace Cuisine


In [ ]:
chain.memory

In [ ]:
type(chain.memory)

NoneType

#### ConversationBufferMemory

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
name = chain.run("Mexican")
print(name)



Taco Fiesta


In [ ]:
name = chain.run("Arabic")
print(name)



Al-Fez Restaurant


In [ ]:
print(chain.memory.buffer)

Human: Mexican
AI: 

Taco Fiesta
Human: Arabic
AI: 

Al-Fez Restaurant


#### ConversationChain

In [ ]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [ ]:
convo.run("Who won the first cricket world cup?")

' The first Cricket World Cup was won by the West Indies in 1975.'

In [ ]:
convo.run("How much is 5+5?")

' 10.'

In [ ]:
convo.run("Who was the captain ofthe winning team?")

' The captain of the West Indies team that won the first Cricket World Cup in 1975 was Clive Lloyd.'

In [ ]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup?
AI:  The first Cricket World Cup was won by the West Indies in 1975.
Human: How much is 5+5?
AI:  10.
Human: Who was the captain ofthe winning team?
AI:  The captain of the West Indies team that won the first Cricket World Cup in 1975 was Clive Lloyd.


#### ConversationBufferWindowMemory

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)

convo = ConversationChain(
    llm=OpenAI(temperature=0.7),
    memory=memory
)
convo.run("Who won the first cricket world cup?")

' The first Cricket World Cup was held in 1975 and the winner was the West Indies.'

In [ ]:
convo.run("How much is 5+5?")

' 5+5 is 10.'

In [ ]:
convo.run("Who was the captain of the winning team?")

" I'm sorry, I don't know."